# Loss functions used.

## FFNN

In [2]:
#Quadratic loss on call option
def loss_call(y_true,y_pred):
    if TC==True:
        TransactionCost=Proportional_cost(y_true,y_pred)
    else:
        TransactionCost=0
    return (callprice + kb.sum(y_pred * y_true,axis=-1) - kb.maximum(S0 + kb.sum(y_true[:,::2],axis=-1) - K,0.)-TransactionCost)**2

def loss_digi_call(y_true,y_pred):
    if TC==True:
        TransactionCost=Proportional_cost(y_true,y_pred)
    else:
        TransactionCost=0
    hedge_pf_payoff = kb.sum(y_pred * y_true,axis=-1)
    option_payoff = kb.maximum(S0 + kb.sum(y_true[:,::2],axis=-1) - K,0.)/(S0+kb.sum(y_true[:,::2],axis=-1)-K)
    return (digiprice + hedge_pf_payoff - option_payoff - TransactionCost)**2

def loss_call_spread(y_true,y_pred):
    if TC==True:
        TransactionCost=Proportional_cost(y_true,y_pred)
    else:
        TransactionCost=0
    hedge_pf_payoff = kb.sum(y_pred * y_true,axis=-1)
    call_long = kb.maximum(S0 + kb.sum(y_true[:,::2],axis=-1) - K1,0.)
    call_short = kb.maximum(S0 + kb.sum(y_true[:,::2],axis=-1) - K2,0.)
    option_payoff = (call_long-call_short)/(K2-K1)
    return (callspread_price + hedge_pf_payoff - option_payoff - TransactionCost)**2

In [1]:
#transaction costs
def Proportional_cost(dS,deltas):#we could also call the input y_pred,y_true,epsilon
    
    #finding changes to pf.
    delta1=deltas[:,::2]
    delta2=deltas[:,1::2]
    
    #for initial TC
    delta1_t0=delta1[:,0]
    delta2_t0=delta2[:,0]
    
    #for terminal TC
    delta1_tT=delta1[:,steps-1]
    delta2_tT=delta2[:,steps-1]
    
    #for intermediate TC
    #finding changes to pf.
    delta1_I=kb.abs(tf.subtract(delta1[:,1:],delta1[:,:-1]))
    delta2_I=kb.abs(tf.subtract(delta2[:,1:],delta2[:,:-1]))
    
    #finding asset prices.
    dS1=dS[:,::2]
    dS2=dS[:,1::2]
    S1=tf.math.cumsum(dS1)+S0
    S2=tf.math.cumsum(dS2)+L(0,V0)#L function is initial price of Swap
    
    #for initial TC
    S1_t0=S0
    S2_t0=L(0,V0)
    
    #for terminal TC
    S1_tT=S1[:,steps-1]
    S2_tT=S2[:,steps-1]
    
    #for intermediate TC.
    #(We remove the price in the last time step)
    S1_I=tf.slice(S1,[0,0],[S1.get_shape()[0],steps-1])
    S2_I=tf.slice(S2,[0,0],[S2.get_shape()[0],steps-1])
    
    #initial TC
    TCS1_0=epsilon*kb.abs(delta1_t0)*S1_t0
    TCS2_0=epsilon*kb.abs(delta2_t0)*S2_t0
    
    #Terminal TC
    TCS1_T=epsilon*kb.abs(delta1_tT)*S1_tT
    TCS2_T=epsilon*kb.abs(delta2_tT)*S2_tT
    
    #Intermediate TC
    TCS1_I=epsilon*kb.sum(delta1_I*S1_I,axis=1)
    TCS2_I=epsilon*kb.sum(delta2_I*S2_I,axis=1)
    
    Cost=TCS1_0+TCS2_0+TCS1_I+TCS2_I+TCS1_T+TCS2_T
    return Cost

#¤consider implementing other types of transacton cost

In [1]:
#ES - not used
def loss_ES(y_true,y_pred):
    if TC==True:
        TransactionCost=kb.mean(Proportional_cost(y_true,y_pred))
    else:
        TransactionCost=0
    #we have a minus in front b/c we need the loss
    pf_loss=-(callprice + kb.sum(y_pred * y_true,axis=-1) - kb.maximum(S0 + kb.sum(y_true[:,::2],axis=-1) - K,0.)-TransactionCost)#do we need a minus here?
    quantile,idx=tf.math.top_k(pf_loss,tf.cast((1-alpha)*pf_loss.shape[0], tf.int32))
    ES_alphas=kb.mean(quantile)
    #sortet = tf.sort(pf_loss)
    #cutoff = int(tfnp.floor(alpha*sortet.shape[0]))
    #quantile = sortet[:cutoff]
    #ES_alpha = -kb.mean(quantile)
    return ES_alpha#tf.repeat(ES_alpha,pf_loss.shape[0])#not sure this is supposed to be like this.

## RNN

In [ ]:
def loss_call_RNN(y_true,y_pred):
    if TC==True:
        TransactionCost=Proportional_cost_RNN(y_true,y_pred)
    else:
        TransactionCost=0
    return (callprice + kb.sum(y_pred * y_true,axis=(1,2)) - kb.maximum(S0 + kb.sum(y_true[:,:,::2],axis=(1,2)) - K,0.)-TransactionCost)**2

In [ ]:
def Proportional_cost_RNN(dS,deltas):#we could also call the input y_pred,y_true,epsilon
    
    #finding changes to pf.
    delta1=deltas[:,:,0]#changed
    delta2=deltas[:,:,1]#changed
    
    #for initial TC
    delta1_t0=delta1[:,0]#ok
    delta2_t0=delta2[:,0]#ok
    
    #for terminal TC
    delta1_tT=delta1[:,steps-1]#ok
    delta2_tT=delta2[:,steps-1]#ok
    
    #for intermediate TC
    #finding changes to pf.
    delta1_I=kb.abs(tf.subtract(delta1[:,1:],delta1[:,:-1]))#ok
    delta2_I=kb.abs(tf.subtract(delta2[:,1:],delta2[:,:-1]))#ok
    
    #finding asset prices.
    dS1=dS[:,:,0]#changed
    dS2=dS[:,:,1]#changed
    S1=tf.math.cumsum(dS1)+S0#ok
    S2=tf.math.cumsum(dS2)+L(0,V0)#L function is initial price of Swap #ok
    
    #for initial TC
    S1_t0=S0#ok
    S2_t0=L(0,V0)#ok
    
    #for terminal TC
    S1_tT=S1[:,steps-1]#ok
    S2_tT=S2[:,steps-1]#ok
    
    #for intermediate TC.
    #(We remove the price in the last time step)
    S1_I=tf.slice(S1,[0,0],[S1.get_shape()[0],steps-1])#ok
    S2_I=tf.slice(S2,[0,0],[S2.get_shape()[0],steps-1])#ok
    
    #initial TC
    TCS1_0=epsilon*kb.abs(delta1_t0)*S1_t0#ok
    TCS2_0=epsilon*kb.abs(delta2_t0)*S2_t0#ok
    
    #Terminal TC
    TCS1_T=epsilon*kb.abs(delta1_tT)*S1_tT#ok
    TCS2_T=epsilon*kb.abs(delta2_tT)*S2_tT#ok
    
    #Intermediate TC
    TCS1_I=epsilon*kb.sum(delta1_I*S1_I,axis=1)#ok
    TCS2_I=epsilon*kb.sum(delta2_I*S2_I,axis=1)#ok
    
    Cost=TCS1_0+TCS2_0+TCS1_I+TCS2_I+TCS1_T+TCS2_T
    return Cost

#consider implementing other types of transacton cost

In [ ]:
#not used
def loss_ES_RNN(y_true,y_pred):
    if TC==True:
        TransactionCost=kb.mean(Proportional_cost_RNN(y_true,y_pred))
    else:
        TransactionCost=0
    pf_loss = (callprice + kb.sum(y_pred * y_true,axis=(1,2)) - kb.maximum(S0 + kb.sum(y_true[:,:,::2],axis=(1,2)) - K,0.)-TransactionCost)
    sortet = tf.sort(pf_loss)
    cutoff = int(tfnp.floor(alpha*sortet.shape[0]))
    quantile = sortet[:cutoff]
    ES_alpha = -kb.mean(quantile)